In [1]:
import os
import torch
import subprocess as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split


In [2]:
base_path = sp.getoutput('git rev-parse --show-toplevel')
os.chdir(base_path)

from src import embed

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nuriaadellraventos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Data Preparation

In [3]:
# Load data
data_dir = 'data'
file = 'utterances_clean2014-2018.csv'
df_raw = pd.read_csv(os.path.join(base_path, data_dir, file))

/var/folders/1r/lbtr12kn1_x17pfbmlxcjxf40000gn/T/ipykernel_37226/2014623051.py:4: DtypeWarning: Columns (17,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(os.path.join(base_path, data_dir, file))


In [4]:
# In the context of pandas' groupby and agg methods, 
# 'first' is an aggregation function that returns the 
# first non-null value in each group of values.
df = (df_raw.groupby('case_id')
        .agg({'text': ' '.join, 'win_side': 'first'})
        .reset_index()
        .drop(['case_id'], axis=1))

df.head()

,text,win_side
0,We will hear argument first this morning in Ca...,1.0
1,"We'll hear argument next in Case 12-1497, Kell...",1.0
2,"We will hear argument next in Case 131010, M&G...",1.0
3,We'll hear argument first this morning in Case...,1.0
4,We will hear argument first this morning in Ca...,1.0


In [12]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=123)

### BoW Embedding

In [14]:
# Bag of Words
vocab = embed.get_vocab(train_df)
vocab_size = len(vocab)
vocab_size

1993

### Create training, validation, and testing dataloaders

In [29]:
BATCH_SIZE = 64

train_valid_data = list(train_df)
num_train = int(len(train_valid_data) * 0.95)
num_valid = len(train_valid_data) - num_train
train_data, valid_data = random_split(
    train_valid_data, [num_train, num_valid])

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE,
                              shuffle=True, 
                              collate_fn=lambda batch: embed.collate_into_bow(batch, vocab)) # pass vocab to collate function
valid_dataloader = DataLoader(valid_data, batch_size=BATCH_SIZE,
                              shuffle=False, 
                              collate_fn=lambda batch: embed.collate_into_bow(batch, vocab))
test_dataloader = DataLoader(test_df, batch_size=BATCH_SIZE,
                             shuffle=False, 
                             collate_fn=lambda batch: embed.collate_into_bow(batch, vocab))